In [ ]:
#clone the GTSRB dataset
!git clone https://bitbucket.org/jadslim/german-traffic-signs

In [ ]:
#listing the contents of the dataset
!ls german-traffic-signs/

In [ ]:
#load the csv file
import pandas as pd
df = pd.read_csv('german-traffic-signs/signnames.csv')
df.head(5)

In [ ]:
#importing packages 
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils.np_utils import to_categorical
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import pickle
import random 
import pandas as pd
import cv2

In [ ]:
#load train,test and validation datasets
np.random.seed(0)
with open('german-traffic-signs/train.p', 'rb') as f:
    train_data = pickle.load(f)
with open('german-traffic-signs/test.p', 'rb') as f:
    test_data = pickle.load(f)
with open('german-traffic-signs/valid.p', 'rb') as f:
    valid_data = pickle.load(f)

In [ ]:
train_x, train_y = train_data['features'],train_data['labels']
val_x, val_y = valid_data['features'],valid_data['labels']
test_x,test_y = test_data['features'],test_data['labels']

print('Train data shape',train_x.shape,'Train data label',train_y.shape)
print('Test data shape',test_x.shape,'Test data label',test_y.shape)
print('valid data shape',val_x.shape,'Valid data label',val_y.shape)

In [ ]:
#getting number of samples from each class in training dataset
import os
num_samples = []
for i in range(len(df)):
    file_count = len(os.listdir('GTSRB/Train/'+str(i)+'/'))
    num_samples.append(file_count)
print(len(num_samples))

In [ ]:
print(num_samples)

In [ ]:
# plot the distrubution of the number of samples from each class in training dataset
plt.figure(figsize=(12,4))
plt.bar(range(0, len(df)), num_samples)
plt.title('distribution of the dataset')
plt.xlabel('class_labels')
plt.ylabel('number of images')
plt.show()

In [ ]:
plt.imshow(train_x[3000])

In [ ]:
# Converting the image into grayscale
def grayscale(image):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    return image

img = grayscale(train_x[3000])
plt.imshow(img, cmap = plt.get_cmap('gray'))

In [ ]:
# Using  histogram equalization technique to standardize contrast/light in all images
def equalize(image):
    image = cv2.equalizeHist(image)
    return image
img = equalize(img)
plt.imshow(img, cmap=plt.get_cmap('gray'))

In [ ]:
def processing(image):
    gray_img = grayscale(image)
    img = equalize(gray_img) 
    img = img/255
    return img

In [ ]:
#Convert all train, test and validation dataset to grayscale & histogram technique

train_x = np.asarray(list(map(processing, train_x)))
val_x = np.asarray(list(map(processing, val_x)))
test_x = np.asarray(list(map(processing, test_x)))

In [ ]:
plt.imshow(train_x[random.randint(0, len(train_x-1))],cmap=plt.get_cmap('gray'))

In [ ]:
train_x.shape[0]

In [ ]:
# Reshape data to 3 dimensional
# train_x.shape[0] -> number of images
# train_x.shape[1] -> height
# train_x.shape[2] -> width
# 1 -> dimension of the image and 1 denotes grayscale image and for RGB it is 3

train_x = train_x.reshape(train_x.shape[0],train_x.shape[1],train_x.shape[2],1)
val_x = val_x.reshape(val_x.shape[0],val_x.shape[1],val_x.shape[2],1)
test_x = test_x.reshape(test_x.shape[0],test_x.shape[1],test_x.shape[2],1)


In [ ]:
# Generate some datasets with different angles using Keras Imagedatagenerator

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(width_shift_range=0.1,
                            height_shift_range=0.1,
                            zoom_range=0.2,
                            shear_range=0.1,
                            rotation_range=10.)
datagen.fit(train_x)
#batches = datagen.flow(train_x,train_y,batch_size=15)


In [ ]:
# One-hot encode the labels

train_y = to_categorical(train_y,43)
test_y = to_categorical(test_y,43)
val_y = to_categorical(val_y,43)

In [ ]:
#Define the model

from keras.layers import ReLU
def build_model():
    model = Sequential()
    model.add(Conv2D(60,(5,5),input_shape=(train_x.shape[1],train_x.shape[2],train_x.shape[3])))
    model.add(ReLU())
    model.add(Conv2D(60,(5,5)))
    model.add(ReLU())
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(30,(3,3)))
    model.add(ReLU())
    model.add(Conv2D(30,(3,3)))
    model.add(ReLU())
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Flatten())
    model.add(Dense(500,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(df['ClassId']),activation='softmax'))
    
    model.compile(Adam(lr=0.001), loss="categorical_crossentropy",metrics=["accuracy"])
    return model
    

In [ ]:
model = build_model()
print(model.summary())

In [ ]:
#train and save the model
history = model.fit_generator(datagen.flow(train_x,train_y,batch_size=64),
                             steps_per_epoch=150,
                             epochs=28,
                             validation_data=(val_x,val_y),shuffle=1)
# Save the trained model
model.save('model.h5')


###### able to achieve 92% accuracy

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training','validation'])
plt.title('Model loss')
plt.xlabel('epoch')
plt.ylabel('loss')

In [ ]:

# Evaluate model on test data
from sklearn.metrics import precision_recall_fscore_support


score = model.evaluate(test_x, test_y, verbose=0)
test_loss = score[0]
test_accuracy = score[1]

# Make predictions on test data
y_pred = model.predict(test_x)
y_pred_bool = np.argmax(y_pred, axis=1)

# Calculate precision, recall and f1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(np.argmax(test_y, axis=1), y_pred_bool, average='weighted')

print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)
print("Test precision:", precision)
print("Test recall:", recall)
print("Test F1-score:", f1_score)

# Plot accuracy, precision and recall over epochs
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
# plt.plot(history.history['precision'])
# plt.plot(history.history['recall'])
plt.title('Model Performance')
plt.ylabel('Performance')
plt.xlabel('Epoch')
plt.legend(['train_accuracy', 'val_accuracy', 'precision', 'recall'], loc='lower right')
plt.show()


: 

In [ ]:
history = model.fit(train_x, train_y, epochs=10, validation_data=(val_x, val_y))


In [ ]:
import requests
from PIL import Image
url = 'https://c8.alamy.com/comp/A0RX23/cars-and-automobiles-must-turn-left-ahead-sign-A0RX23.jpg'
r = requests.get(url, stream=True)
img = Image.open(r.raw)
plt.imshow(img)

In [ ]:
img = np.asarray(img)
img = cv2.resize(img,(32,32))
img = processing(img)
plt.imshow(img, cmap=plt.get_cmap('gray'))

In [ ]:
df.columns

In [ ]:
img = img.reshape(1,32,32,1)
label = model.predict_classes(img)
print('Predicted class is ',str(label))
print('The traffic sign is ',df['SignName'][int(label)])